In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras import layers
import re
from simpletransformers.classification import ClassificationModel, ClassificationArgs


In [2]:
import numpy as np
def dummy_npwarn_decorator_factory():
  def npwarn_decorator(x):
    return x
  return npwarn_decorator
np._no_nep50_warning = getattr(np, '_no_nep50_warning', dummy_npwarn_decorator_factory)

In [3]:
df = pd.read_csv('mbti_1.csv')

### Text Cleaning and Preprocessing

In [6]:
# Text Cleaning 
# remove links from posts
def remove_links(text):
  url_pattern = r'https?://\S+|www\.\S+'
  return re.sub(url_pattern, '', text)

# only get letters in text
def get_alphanumeric(text):
    return re.sub(r'[^a-zA-Z\s]', '', text)

In [9]:
df['posts'] = df['posts'].apply(remove_links).apply(get_alphanumeric).str.lower()
df = df.assign(i_e = df['type'].apply(lambda x: 'I' in x))
df = df.assign(s_n = df['type'].apply(lambda x: 'S' in x))
df = df.assign(t_f = df['type'].apply(lambda x: 'T' in x))
df = df.assign(j_p = df['type'].apply(lambda x: 'J' in x))
df

,type,posts,i_e,s_n,t_f,j_p
0,INFJ,and intj moments sportscenter not top ten ...,True,False,False,True
1,ENTP,im finding the lack of me in these posts very ...,False,False,True,False
2,INTP,good one course to which i say i know tha...,True,False,True,False
3,INTJ,dear intp i enjoyed our conversation the oth...,True,False,True,True
4,ENTJ,youre firedthats another silly misconception t...,False,False,True,True
...,...,...,...,...,...,...
8670,ISFP,just because i always think of cats as fi dom...,True,True,False,False
8671,ENFP,soif this thread already exists someplace else...,False,False,False,False
8672,INTP,so many questions when i do these things i wo...,True,False,True,False
8673,INFP,i am very conflicted right now when it comes t...,True,False,False,False


In [16]:
df['i_e']

0        True
1       False
2        True
3        True
4       False
        ...  
8670     True
8671    False
8672     True
8673     True
8674     True
Name: i_e, Length: 8675, dtype: bool

### Model

In [4]:
# Optional model configuration
model_args = ClassificationArgs(num_train_epochs=1)

# Create a ClassificationModel
model = ClassificationModel('roberta', 'roberta-base', use_cuda=False, args={'reprocess_input_data': True, 'overwrite_output_dir': True})

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [11]:
model.train_model(df[['posts', 'i_e']])


/Users/calwinli/miniforge3/envs/ml/lib/python3.10/site-packages/simpletransformers/classification/classification_model.py:610: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  warnings.warn(


  0%|          | 0/17 [00:00<?, ?it/s]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Running Epoch 1 of 1:   0%|          | 0/1085 [00:00<?, ?it/s]

(1085, 0.5352343044797396)

In [ ]:
preds, raw = model.predict(list(df['posts'].values))

  0%|          | 0/17 [00:00<?, ?it/s]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

  0%|          | 0/87 [00:00<?, ?it/s]

In [24]:
from simpletransformers.classification import ClassificationModel


model = ClassificationModel(
    "roberta", "roberta-base"
)

ValueError: 'use_cuda' set to True when cuda is unavailable. Make sure CUDA is available or set use_cuda=False.